In [ ]:
onnx_model_path = "./model/model.onnx"
ExecutionProvider="QNNExecutionProvider"
provider_options={}
if ExecutionProvider == "QNNExecutionProvider":
    provider_options={"backend_path": "QnnHtp.dll"}
elif ExecutionProvider == "OpenVINOExecutionProvider":
    onnx_model_path = "./model/ov_model.onnx"

In [ ]:
import onnxruntime as ort